# Deploy a Grafana Dashboard
To track the different stocks on a live dashboard we will use **Grafana**.<br>
We will use [Grafwiz](https://github.com/v3io/grafwiz) to define and deploy the dashboard directly from this notebook

In [1]:
# install prerequsits 
# !pip install git+https://github.com/v3io/grafwiz --upgrade
# !pip install v3io_frames

In [2]:
import mlrun
project = mlrun.get_or_create_project(name='stocks',user_project=True, context="src/")

> 2022-09-04 15:42:57,642 [info] loaded project stocks from MLRun DB


In [3]:
import grafwiz
import os
import v3io_frames as v3f

framesd = os.getenv("V3IO_FRAMESD",'framesd:8081')
client = v3f.Client(framesd, container=os.getenv('V3IO_CONTAINER', 'projects'))

grafwiz.DataSource(name='iguazio').deploy('http://grafana',use_auth=True)

dash = grafwiz.Dashboard("stocks", start='now-7d', dataSource='iguazio')

df_columns = ['datetime', 'prediction', 'tickers', 'true']
news_kv = '/stocks-'+ os.environ['V3IO_USERNAME'] + '/artifacts/stocks_prediction'
client.execute("kv",table = news_kv, command = "infer")

tbl = grafwiz.Table(title='Current news Value', span=12,columns=df_columns).source(table=news_kv,fields=df_columns,container='projects')
dash.row([tbl])

dash.deploy('http://grafana')

Datasource iguazio already exists
Datasource iguazio created successfully
Dashboard stocks created successfully


## Adding stocks_serving function scheduler

In [4]:
fn = project.get_function('serving_stocks')
serving_address = fn.spec.command

In [6]:
%%writefile src/invoker.py

import requests
import datetime

def ingestion_service_invoker(endpoint):     
    t = requests.post(endpoint,json={"vector_name": 'stocks',
                                          "start_time" : 59,
                                          "end_time"   : 0,
                                          "seq_size"   : 5})

Overwriting src/invoker.py


In [8]:
# specifying '0 8 * * *' as schedule will trigger the function every day at 08:00 AM
fn = mlrun.code_to_function(name='prediction_kv',kind='job',image='mlrun/mlrun',handler='ingestion_service_invoker', filename='src/invoker.py')
fn.run(params={'endpoint':serving_address}, schedule='0 */1 * * *')

> 2022-09-04 15:43:35,969 [info] starting run prediction-kv-ingestion_service_invoker uid=b4b5d0bb9b494796909ba0f7dd7dc4b7 DB=http://mlrun-api:8080
> 2022-09-04 15:43:36,256 [info] task scheduled, {'schedule': '0 */1 * * *', 'project': 'stocks-dani', 'name': 'prediction-kv-ingestion_service_invoker'}
